# AEMET GRID

In [4]:
from datetime import datetime
from os.path import splitext, join, basename, dirname
import pandas as pd
import numpy as np

## Rejillas

Master files from aemet

In [13]:
maestros = (r'E:\db02\aemet\Serie_AEMET_v2_pcp_1951a202006_txt\maestro_red_hr_SPAIN.txt',
            r'E:\db02\aemet\Serie_AEMET_v1_tmin_1951a202006_txt\maestro_red_hr_tmin.txt',
            r'E:\db02\aemet\Serie_AEMET_v1_tmax_1951a202006_txt\maestro_red_hr_tmax.txt')
dir_out = r'E:\IGME20\20201215_aquifer\_data\aemet_grid'

In [20]:
for f0 in maestros:
    print(f0)
    df = pd.read_csv(f0, sep='\s+')
    print(df.head())
    print(df.shape)

E:\db02\aemet\Serie_AEMET_v2_pcp_1951a202006_txt\maestro_red_hr_SPAIN.txt
   1  -2.948  35.307   102.
0  2  -5.323  35.898   58.0
1  3  -5.630  36.049  124.0
2  4  -5.568  36.049  186.0
3  5  -5.507  36.049  163.0
4  6  -5.815  36.100   49.0
(16155, 4)
E:\db02\aemet\Serie_AEMET_v1_tmin_1951a202006_txt\maestro_red_hr_tmin.txt
   1  -2.948  35.307   102.
0  2  -5.323  35.898   58.0
1  3  -5.630  36.049  124.0
2  4  -5.568  36.049  186.0
3  5  -5.507  36.049  163.0
4  6  -5.815  36.100   49.0
(16155, 4)
E:\db02\aemet\Serie_AEMET_v1_tmax_1951a202006_txt\maestro_red_hr_tmax.txt
   1  -2.948  35.307   102.
0  2  -5.323  35.898   58.0
1  3  -5.630  36.049  124.0
2  4  -5.568  36.049  186.0
3  5  -5.507  36.049  163.0
4  6  -5.815  36.100   49.0
(16155, 4)


Now I change the format to csv

In [25]:
for f0, new_name in zip(maestros, ('maestro_p.csv', 'maestro_tmin.csv', 'maestro_tmax.csv') ):
    file_out = join(dir_out, new_name)
    print(file_out)
    df = pd.read_csv(f0, sep='\s+', header = None, names= ('gid', 'lon', 'lat', 'z'), encoding='utf-8')
    df.to_csv(file_out, index=False, encoding='utf-8', float_format='%.2f')
print('fin')

E:\IGME20\20201215_aquifer\_data\aemet_grid\maestro_p.csv
E:\IGME20\20201215_aquifer\_data\aemet_grid\maestro_tmin.csv
E:\IGME20\20201215_aquifer\_data\aemet_grid\maestro_tmax.csv
fin


## Data

### grid poins en Campo de Cartagena + Triásico de los Victorias con un buffer de 1 km dissolved

In [49]:
df_points = pd.read_csv(r'E:\IGME20\20201215_aquifer\qgis\aemet_grid\lyrs\selected_points.csv', delimiter=",", converters = {'gid': np.int64})
print(df_points.columns)
points = [int(row.gid) for index, row in df_points.iterrows()]
print(points)

Index(['gid', 'lat', 'lon', 'z'], dtype='object')
[1877, 1878, 1879, 1880, 1881, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2394, 2395, 2396, 2397, 2398, 2399, 2400, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2597, 2598, 2599, 2600, 2699]


In [50]:
data_files = (r'E:\db02\aemet\Serie_AEMET_v2_pcp_1951a202006_txt\pcp_red_SPAIN_1951-20200630.txt',
              r'E:\db02\aemet\Serie_AEMET_v1_tmin_1951a202006_txt\tmin_red_SPAIN_1951-20200630.txt',
              r'E:\db02\aemet\Serie_AEMET_v1_tmax_1951a202006_txt\tmax_red_SPAIN_1951-20200630.txt')

The process below is time consuming; if the aemet data files are not changed don't run it again

In [52]:
start_time = datetime.now() 
for f1 in data_files:
    print(f1)
    df = pd.read_csv(f1, sep='\s+')
    print(df.head())
    print(df.shape) 
print('Time elapsed (hh:mm:ss) {}'.format(datetime.now() - start_time))

E:\db02\aemet\Serie_AEMET_v2_pcp_1951a202006_txt\pcp_red_SPAIN_1951-20200630.txt
   19510101  0.37   8.05  17.80  17.89  16.11  15.42  15.96  16.79  18.46  \
0  19510102  4.29  18.46   8.65   9.47  10.30   6.03   5.98   6.18   6.86   
1  19510103  0.00   1.55   0.00   0.93   1.65   0.00   0.00   0.00   0.89   
2  19510104  0.00   0.32   2.01   1.99   1.92   2.02   2.16   2.27   2.30   
3  19510105  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   
4  19510106  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   

   ...  24.89  23.23.2  21.89  20.60  25.13.3  24.03  23.06  23.01.1  22.63.1  \
0  ...  19.31    19.80  20.05  20.13    14.14  15.59  16.81    17.68    18.37   
1  ...   3.17     2.83   2.55   2.27     0.98   1.04   1.13     1.47     1.68   
2  ...   0.25     0.26   0.25   0.24     0.23   0.30   0.33     0.31     0.30   
3  ...   0.00     0.00   0.00   0.00     0.00   0.00   0.00     0.00     0.00   
4  ...   0.07     0.09   0.10   0.10     0.00   0.0

<b>The points of interest has beeen selected by intersecting the grid points of aemet with the the union of aquifers CC + TV with a 1km buffer<br>
and manually adding other points near the buffer</b><br>Write the list of points as integers

In [31]:
points = [1877, 1878, 1879, 1880, 1881, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2394, 2395, 2396, 2397, 2398, 2399, 2400, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2597, 2598, 2599, 2600, 2699]
points.sort()
print(points)
print(len(points))

[1877, 1878, 1879, 1880, 1881, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2394, 2395, 2396, 2397, 2398, 2399, 2400, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2597, 2598, 2599, 2600, 2699]
54


<b>A new set of centroids to extract after the above set</b><br>Observe that the name of the list is the same

In [10]:
points = [2393, 2596, 2698, 1882, 2084]
points.sort()
print(points)
print(len(points))

[1882, 2084, 2393, 2596, 2698]
5


Path to the 3 original files of aemet. I avoid inserting the whole content of the files in the database

In [6]:
data_files = (r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\pcp_red_SPAIN_1951-20200630.txt',
              r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\tmin_red_SPAIN_1951-20200630.txt',
              r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\tmax_red_SPAIN_1951-20200630.txt')

<b>I open the aemet file using the data_files but reading only the columns of points inside zone of interest<br>
Here I open aemet file with pandas and save it to a csv</b><br>
If you have a prior file it will ve overwritted; change the prior file name if you want to preserver it 

In [11]:
start_time = datetime.now() 
headers = [0] + points
for f2 in data_files:
    print(f2)
    df = pd.read_csv(f2, sep='\s+', header = None, names= headers, usecols= headers, encoding='utf-8', converters={0: str})
    name, extension = splitext(f2)
    output = name + '.csv'
    df.to_csv(output, index=False, encoding='utf-8')
print('Time elapsed (hh:mm:ss) {}'.format(datetime.now() - start_time))

E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\pcp_red_SPAIN_1951-20200630.txt
E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\tmin_red_SPAIN_1951-20200630.txt
E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\tmax_red_SPAIN_1951-20200630.txt
Time elapsed (hh:mm:ss) 0:08:08.513960


Now I change the csv aemet format with the selected columns (points) to a new ccv format in wich each row is the point, date and variable (pec, tmin, tmax<br>
Each variable is writed in is own file (total 3 files) 

In [13]:
start_time = datetime.now() 
files = (r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\pcp_red_SPAIN_1951-20200630.csv',
         r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\tmin_red_SPAIN_1951-20200630.csv',
         r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\data\tmax_red_SPAIN_1951-20200630.csv')
dir_out = r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\output'

with open(files[0], 'r') as fi:
    for i, line in enumerate(fi):
        if i == 0:
            headers = line.strip().split(',')
            break

for file, newname in zip(files, ('serie_p', 'serie_tmin', 'serie_tmax')):
    output = join(dir_out, f'{newname}.csv')
    print(output)
    with open(file, 'r') as fi, open(output, 'w') as fo:
        for i, line in enumerate(fi):
            if i == 0:
                fo.write('"gid","fecha","value"\n')
                continue
            words = line.strip().split(',')
            for h1, w1 in zip(headers[1:], words[1:]):
                fecha = f'{words[0][0:4]}-{words[0][4:6]}-{words[0][6:8]}'
                fo.write(f'{h1},{fecha},{w1}\n')
print('Time elapsed (hh:mm:ss) {}'.format(datetime.now() - start_time))

E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\output\serie_p.csv
E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\output\serie_tmin.csv
E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\output\serie_tmax.csv
Time elapsed (hh:mm:ss) 0:00:02.479330


Finally I merge the 3 files in one with structure of the table if the db

In [15]:
start_time = datetime.now() 
files_merge = [r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\output\serie_p.csv',
               r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\output\serie_tmin.csv',
               r'E:\IGME20\20201215_aquifer\_data\aemet_grid\originales\output\serie_tmax.csv']
file_ouput = join(dir_out,'p_tmin_tmax_cctv.csv')
with open(files_merge[0], 'r') as f0, open(files_merge[1], 'r') as f1, open(files_merge[2], 'r') as f2, open(file_ouput, 'w') as fo:
    fo.write('gid,fecha,pmm,tmingc,tmaxgc\n')
    for i, (line0, line1, line2) in enumerate(zip(f0, f1, f2)):
        if i == 0:
            continue
        line0 = line0.strip()
        tmin = line1.strip().split(',')[-1]
        tmax = line2.strip().split(',')[-1]
        fo.write(f'{line0},{tmin},{tmax}\n')
print('Time elapsed (hh:mm:ss) {}'.format(datetime.now() - start_time))

Time elapsed (hh:mm:ss) 0:00:03.555424


This last file is imported to the db with the psql command